# Analysis

## Get backend data

In [6]:
import os, re, sys
sys.path.append(r"C:\Users\robin\Documents\projets\detection_nappe_hydrocarbures_IMT_cefrem")
sys.path.append(r"C:\Users\robin\Documents\projets\detection_nappe_hydrocarbures_IMT_cefrem\main")
sys.path.append(r"C:\Users\robin\Documents\projets\detection_nappe_hydrocarbures_IMT_cefrem\main\src")
sys.path.append(r"C:\Users\robin\Documents\projets")
from main.FolderInfos import FolderInfos
import pandas as pd
import json

from main.src.analysis.analysis.metadata.CacheMetadataManager import CacheMetadataManager
from main.src.analysis.analysis.metadata.Formatters.Formatter import Formatter
from main.src.analysis.analysis.metrics.MetricManager import MetricManager
from main.src.analysis.analysis.exceptions import ElementNotFound

FolderInfos.init(test_without_data=True)
list_dataout_folders = os.listdir(FolderInfos.data_folder)
with open(FolderInfos.root_folder+"main"+FolderInfos.separator+"src"+FolderInfos.separator+"analysis"+FolderInfos.separator+"extract_data.json","r") as fp:
    dico_access = json.load(fp)
dico_metrics = {}
available_metrics = dico_access["metrics"].keys()
runs_metadata = []
with open(FolderInfos.data_folder+"2021-07-03_15h41min41s_"+FolderInfos.separator+"2021-07-03_15h41min41s_parameters.json","r") as cache_infos:
    dico_cache_additional_infos = json.load(cache_infos)
for f in list_dataout_folders:
    if f == "2021-07-18_23h25min14s_":
        b=0
    path_json = FolderInfos.data_folder+f+FolderInfos.separator+f+"parameters.json"
    if os.path.exists(path_json) is True:
        try:
            with open(path_json,"r") as fp:
                dico = json.load(fp)
        except:
            continue
        dico_run_metadata = {"folder_id":f}
        try:
            for metadata_name,dico_metadata in dico_access["metadata"].items():
                formatter = Formatter()
                cache_metric_manager = CacheMetadataManager(dico_access["metadata"],dico,dico_cache_additional_infos,formatter)
                dico_run_metadata[metadata_name] = cache_metric_manager.get(metadata_name)


            metrics = {}
            metric_manager = MetricManager(dico_access["metrics"],dico)
            for metric_name in dico_access["metrics"].keys():
                metrics[metric_name] = metric_manager.get(metric_name)
        except ElementNotFound:
            continue
        dico_metrics[f] = metrics
        runs_metadata.append(dico_run_metadata)

df:pd.DataFrame = pd.DataFrame(runs_metadata)
# import qgrid
# widget = qgrid.show_grid(df_filtered)
# tabloo.show(df_filtered)
print("👍 Backend data ready !")
l_path = FolderInfos.root_folder.split(FolderInfos.separator)[:-1] + ["main","src","analysis","data_taboo.pkl"]
path_cache = FolderInfos.separator.join(l_path)
df.to_pickle(path_cache)
script_path  = FolderInfos.separator.join(FolderInfos.root_folder.split(FolderInfos.separator)[:-1] + ['main','src','analysis','launch_taboo.py'])
print(f"To see runs availables launch the launch_taboo.py script\nat {script_path} . \n"+
        "⚠️ the python launch_taboo.py process needs to be killed \nonce you do not need to visualize anymore the dataframe. "
        "\nClosing the web navigator window is not sufficient !")

👍 Backend data ready !
To see runs availables launch the launch_taboo.py script
at C:\Users\robin\Documents\projets\detection_nappe_hydrocarbures_IMT_cefrem\main\src\analysis\launch_taboo.py . 
⚠️ the python launch_taboo.py process needs to be killed 
once you do not need to visualize anymore the dataframe. 
Closing the web navigator window is not sufficient !


In [7]:
import ipywidgets as widgets
from IPython.display import display,clear_output
from bs4 import BeautifulSoup

html_template = lambda content:"""
<!DOCTYPE html>
<html>
<head>
<meta charset="UTF-8">
<style>
    table {
        border-collapse: collapse;
        margin: 25px 0;
        font-size: 0.9em;
        font-family: sans-serif;
        min-width: 400px;
        box-shadow: 0 0 20px rgba(0, 0, 0, 0.15);
    }
    table thead tr {
        background-color: #009879;
        color: #ffffff;
        text-align: left;
    }
    table th,
    table td {
        padding: 12px 15px;
    }
    table tbody tr {
        border-bottom: 1px solid #dddddd;
    }

    table tbody tr:nth-of-type(even) {
        background-color: rgba(255, 255, 255, 0.25);

    }

    table tbody tr:last-of-type {
        border-bottom: 2px solid #009879;
    }
    table tbody tr.active-row {
        font-weight: bold;
        color: #009879;
    }
</style>
</head>
<body>
%s
</body>
""" % content
out = widgets.Output()
filter_data = widgets.Text(description="Query")
html: widgets.HTML = widgets.HTML()
box = widgets.VBox([filter_data,html])
global df_filtered
with out:
    display(box)
class DataFrameObject:# to solve the bug cannot put dataframe as global
    def __init__(self,df):
        self.orig_df = df
        self.df = None
    def __call__(self, string):
        self.df = self.orig_df.query(string)
        return self.df
data = DataFrameObject(df)
def handler_filter(v):
    df_filtered: pd.DataFrame = data(filter_data.value)
    html_value = df_filtered.to_html()
    html_value = html_template(html_value)
    soup = BeautifulSoup(html_value, 'html.parser')
    html.value = soup.prettify()
filter_data.on_submit(handler_filter)
box



In [8]:
filtered_dataframe = data.df
import ipywidgets as widgets
class PersoToggleButton(widgets.ToggleButton):
    def __init__(self,name=None,*args,**kargs):
        super(PersoToggleButton, self).__init__(*args,**kargs)
        self.name = name

compute_width = lambda value:f"{max(len(str(value)) * 12, 60) + 20}px"
list_vertical_widgets = []
dico_changing_widgets_metadata = {}
list_changing_widgets_metrics = []
list_changing_widgets_tr_valid = []
# Widgets for the metadata ------------------------------------------------------------------
label_metadata_filter = "Metadata"
llocalvertwidgets = []
dico_changing_widgets_reorganized = {"metadata":{},"metrics":{},"mode":{}}
for metadata_name in df.columns:

    if metadata_name == "folder_id":
        continue
    possible_values = df[metadata_name].unique()
    label = widgets.Label(metadata_name)
    dico_changing_widgets_reorganized["metadata"][metadata_name] = {str(value):PersoToggleButton(
        value=True,
        description=str(value),
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='',
        icon='check',
        layout = widgets.Layout(width = compute_width(value), margin='0px 0px 0px 0px'),
        name="metadata|"+metadata_name
    ) for value in possible_values}

    possibilities = list(dico_changing_widgets_reorganized["metadata"][metadata_name].values())
    dico_changing_widgets_metadata[metadata_name] = possibilities
    box = widgets.HBox(layout=widgets.Layout(width='100%',display='inline-flex',flex_flow='row wrap'),width='100%')
    box.overflow_x = 'auto'
    box.children = [label]+possibilities
    llocalvertwidgets.append(box)

box = widgets.VBox(width='100%')
box.children = llocalvertwidgets
list_vertical_widgets.append(box)
# Widgets to select metrics ---------------------------------------------------------------
label_metrics_filter = "Metrics"
dico_changing_widgets_reorganized["metrics"] = {str(metric_name):PersoToggleButton(
        value=True,
        description=metric_name,
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='',
        icon='check',
        layout = widgets.Layout(width = compute_width(metric_name), margin='0px 0px 0px 0px'),
        name="metrics"
    )
    for metric_name in available_metrics}
possibilities = list(dico_changing_widgets_reorganized["metrics"].values())
list_changing_widgets_metrics = possibilities
box = widgets.HBox(layout=widgets.Layout(width='100%',display='inline-flex',flex_flow='row wrap'),width='100%')
box.overflow_x = 'auto'
box.children = possibilities
list_vertical_widgets.append(box)

# widgets to select tr or valid ----------------------------------------------------------
label_tr_valid_filter = "Mode_of_training"
possibilities = [PersoToggleButton(
        value=True,
        description=mode,
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='',
        icon='check',
        layout = widgets.Layout(width = compute_width(mode), margin='0px 0px 0px 0px'),
        name="mode"
    )
    for mode in ["tr","valid"]]
list_changing_widgets_tr_valid = possibilities
box = widgets.HBox(layout=widgets.Layout(width='100%',display='inline-flex',flex_flow='row wrap'),width='100%')
box.overflow_x = 'auto'
box.children = possibilities
list_vertical_widgets.append(box)

titles = ["Metadata", "Metrics","Mode_of_training"]
l = []
for i,[item,title] in enumerate((zip(list_vertical_widgets,titles))):
    accordion = widgets.Accordion(children=[item],titles=(title,))
    accordion.set_title(0,title)
    accordion.selected_index = 0
    if i == 0:
        accordion.selected_index = None
    l.append(accordion)
# widget for moving average
moving_average = widgets.IntText(
    value=100,
    description='mean_window',
    disabled=False
)
# widget to toggle log scale for the y axis
y_log_scale = widgets.ToggleButton(
        value=True,
        description="logy",
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='',
        icon='check',
        layout = widgets.Layout(width = compute_width("logy"), margin='0px 0px 0px 0px')
    )

global_box = widgets.VBox(l+[moving_average,y_log_scale])

# Handlers --------------------------------------------------------------------------------------
from IPython.display import display,clear_output
import plotly.graph_objects as go
import main.src.analysis.tools as tls
import plotly.offline as pyo
pyo.init_notebook_mode()

def filter_metrics(runs):
    global states
    new_dico = {}
    for f,dico in dico_metrics.items():
        if f not in runs:
            continue
        new_dico[f] = {}
        for metric_name,list_values in dico.items():
            if states["metrics"][metric_name] is True:
                new_dico[f][metric_name] = list_values
    return new_dico
def plot(dico_vals,filtered_dataframe):
    global states
    import numpy as np
    list_scatters = []
    import matplotlib.pyplot as plt
    for run,dico_metrics in dico_vals.items():
        line_dataframe = filtered_dataframe[filtered_dataframe["folder_id"] == run]
        batch_size_tr = line_dataframe["tr_batch_size"].values[0]
        batch_size_valid = line_dataframe["valid_batch_size"].values[0]
        for metric_name,metric_values in dico_metrics.items():
            if metric_name == "loss":
                metric_name = "loss_"+filtered_dataframe[filtered_dataframe["folder_id"] == run]["loss"].values[0]
            access_name = "_values"
            for mode,state in states["mode"].items():
                if state is False:
                    continue
                batch_size = batch_size_tr if mode == "tr" else batch_size_valid
                print(mode,"!!!!!!!!!!!!!!",batch_size)
                access_name1 = mode + access_name
                v = metric_values[access_name1]
                window = int(moving_average.value/batch_size)
                new_values = []
                for i in range(len(v)-window):
                    new_values.append(np.mean(v[i:min(i+window,len(v))]))
                v = np.array(new_values)
                x = np.arange(0,v.shape[0]) * batch_size
                x = pd.Series(x)
                v = pd.Series(v)
                graph = go.Scatter(x=x,y=v,mode='lines',name=f"{metric_name} {mode} {run}",
                                   hovertemplate='Sample n°: %{x}'+f'<br>{metric_name} {mode}<br>{run}:'+' %{y}')
                list_scatters.append(graph)
    plt.clf()
    fig = go.Figure(
        data=list_scatters,
        layout=go.Layout(
            title=go.layout.Title(text=""),
            xaxis_title="Number of samples processed",
            yaxis_title="Values",
            legend=dict(
                orientation="h",
                y=1,
                x=1
            ),
            yaxis={'tickformat':'.1e',},
            width=900,
            height = 500
        ),
    )
    if states["others"]["logy"] is True:
        fig.update_yaxes(type="log")
    clear_output()
    display(global_box)
    pyo.iplot(fig)
states = {"others":{"mean_window":moving_average.value,"logy":False},
          "mode":{"tr":False,"valid":False},
          "metrics":{k:False for k in available_metrics},
          "metadata":{k:{
              str(k1):False for k1 in df[k].unique()
          } for k in df.columns}
          }
def handler(v):
    global query
    if v["name"] == "disabled":
        return
    global states
    try:
        name = "others"
        try:
            name = v["owner"].name
        except:
            pass
        name = name.split("|")
        try:
            new_val = not(v["new"]["value"])
        except:
            new_val = not(v["new"])
    except Exception as e:
        return
    if len(name) == 2:
        states[name[0]][name[1]][v["owner"].description] = not(states[name[0]][name[1]][v["owner"].description])
    else:
        states[name[0]][v["owner"].description] = not(states[name[0]][v["owner"].description])
    runs = filtered_dataframe["folder_id"].unique()
    dico_vals = filter_metrics(runs)
    plot(dico_vals,filtered_dataframe)
    try:
        print(dico_vals['2021-06-20_17h39min55s_']["mae"])
    except:
        pass
    print(metric_name)

for name,liste_w in dico_changing_widgets_metadata.items():
    for w in liste_w:
        w.observe(handler)
for w in list_changing_widgets_metrics:
    w.observe(handler)
list_changing_widgets_metrics[-1].value = True
for w in list_changing_widgets_tr_valid:
    w.observe(handler)
for w in [moving_average,y_log_scale]:
    w.observe(handler)
global_box

error_threshold0.5


<Figure size 432x288 with 0 Axes>

Confusion matrix

In [7]:
from main.src.analysis.analysis.metadata.confusion_matrix.backend import ConfusionMatrixBackend
from main.FolderInfos import FolderInfos
import json
FolderInfos.init(test_without_data=True)
folder = "2021-07-22_13h43min53s_"
with open(FolderInfos.data_folder+FolderInfos.separator.join([folder,folder+"parameters.json"]),"r") as fp:
    dico = json.load(fp)
epoch = dico["trainer"]["attr_last_epoch"]
it = dico["trainer"]["attr_last_iter"]
with open(FolderInfos.root_folder+FolderInfos.separator.join(["main","src","analysis","extract_data.json"]),"r") as fp:
    extract_functions_dict = json.load(fp)
value_functions = extract_functions_dict["confusion_matrix"]["value"]
names_functions = extract_functions_dict["confusion_matrix"]["names"]
confusion_matrix = ConfusionMatrixBackend(dico,value_functions,names_functions)
html = confusion_matrix.html
path = FolderInfos.data_folder+FolderInfos.separator.join([folder,folder+f"confusion_matrix_epoch_{epoch}_it_{it}.html"])
with open(path,"w") as fp:
    fp.write(html)


import os
os.system("python -m webbrowser -t "+path)


0